In [9]:
import random
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split

# set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# if you ever use GPU, these help determinism
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.benchmark = False

In [10]:
# converts 'MM:SS' strings to float minutes
def convert_mp(mp):
    if isinstance(mp, str):
        if ':' in mp:
            try:
                mins, secs = mp.split(':')
                return int(mins) + int(secs) / 60
            except:
                return None
        else:
            try:
                return float(mp)
            except:
                return None
    elif isinstance(mp, (int, float)):
        return float(mp)
    else:
        return None


In [11]:
# import data
#data = pd.read_csv('../data-collection/bbref_players_games_simple/g/gilgesh01_Shai_Gilgeous-Alexander_last3.csv')
data = pd.read_csv('../data-collection/bbref_players_games_simple/gilgesh01_Shai_Gilgeous-Alexander_last3_with_opp_stats.csv')
# make sure games are ordered oldest -> newest
if 'Date' in data.columns:
    data = data.sort_values('Date')

# is_home: 1 if home, 0 if away
data['is_home'] = (data['Unnamed: 5'] != '@').astype(float)
# result_win: 1 if team won, 0 if lost
data['result_win'] = data['Result'].str.startswith('W').astype(float)

# features (previous game)
features = [
    'MP', 'FGA', '3PA', 'FTA',
    'FG%', '3P%', '2P%', 'eFG%', 'FT%',
    'TRB', 'AST', 'TOV',
    'GmSc', '+/-', 'PTS', 'is_home', 'result_win',
    'OppOffRtg', 'OppDefRtg', 'OppNetRtg', 'OppPace'
]

# convert MP to float minutes
data['MP'] = data['MP'].apply(convert_mp)

# convert numeric columns
for col in features + ['PTS']:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# create TARGET from *current* game points (before shifting)
current_ppg = 32.4
data['above_ppg'] = (data['PTS'] > current_ppg).astype(float)
target = ['above_ppg']

# regression target: current game points
data['pts_target'] = data['PTS']

# now shift features so they refer to the *previous* game
data[features] = data[features].shift(1)

# drop first row and rows with NaNs
data = data.dropna(subset=features + target + ['pts_target'])

# numpy arrays
X_np = data[features].values
y_np = data[target].values
y_pts_np = data[['pts_target']].values

# chronological split: last 30 games as test
# When holdout_for_eval=True, the last n_test games are kept ONLY for evaluation.
# When holdout_for_eval=False, all games (including those last n_test) are used for training.
holdout_for_eval = False  # set to False when training on all history to predict the next game
n_test = 30
if holdout_for_eval and n_test > 0:
    X_train_np, X_test_np = X_np[:-n_test], X_np[-n_test:]
    y_train_np, y_test_np = y_np[:-n_test], y_np[-n_test:]
    y_pts_train_np, y_pts_test_np = y_pts_np[:-n_test], y_pts_np[-n_test:]
else:
    # train on all available games
    X_train_np, y_train_np = X_np, y_np
    y_pts_train_np = y_pts_np
    # still define a test slice (e.g., last n_test games) for inspection if desired
    if n_test > 0:
        X_test_np, y_test_np = X_np[-n_test:], y_np[-n_test:]
        y_pts_test_np = y_pts_np[-n_test:]
    else:
        # fall back to using the last game as a "test" point
        X_test_np, y_test_np = X_np[[-1]], y_np[[-1]]
        y_pts_test_np = y_pts_np[[-1]]

print(len(X_train_np))
print(len(X_test_np))

# to torch
X_train = torch.tensor(X_train_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.float32)
X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_test = torch.tensor(y_test_np, dtype=torch.float32)
y_pts_train = torch.tensor(y_pts_train_np, dtype=torch.float32)
y_pts_test = torch.tensor(y_pts_test_np, dtype=torch.float32)

150
30


In [12]:
# define model, loss function, and optimizer
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1),
    nn.Sigmoid()
)

# per-sample BCE so we can apply recency weights
criterion = nn.BCELoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

epochs = 1000
lambda_penalty = .8  # tune this (start small like 0.1–0.5)

# recency weights: older games get lower weight, recent games higher
n_train = X_train.shape[0]
idx = torch.arange(n_train, dtype=torch.float32)
# weights in [0.5, 1.0]; adjust endpoints as desired
sample_weights = 0.5 + 0.5 * (idx / (n_train - 1))
sample_weights = sample_weights.view(-1, 1)  # (n_train, 1) to broadcast with probs

for _ in range(epochs):
    optimizer.zero_grad()
    probs = model(X_train)

    # base BCE loss (per-sample), then weighted by recency
    bce_per_sample = criterion(probs, y_train)
    bce = (bce_per_sample * sample_weights).mean()

    # confidence in [0,1]: 0 at 0.5, 1 near 0 or 1
    confidence = torch.abs(probs - 0.5) * 2.0

    # wrong predictions: 1 if wrong, 0 if correct
    hard_preds = (probs >= 0.5).float()
    wrong = torch.abs(hard_preds - y_train)

    # extra penalty:
    #   - all wrong predictions get weighted by confidence AND recency
    #   - WRONG & very high-confidence get an extra (confidence^2) penalty, also recency-weighted
    high_conf_mask = confidence > 0.8
    if high_conf_mask.any():
        high_conf_wrong = (confidence[high_conf_mask] ** 2 *
                           wrong[high_conf_mask] *
                           sample_weights[high_conf_mask]).mean()
    else:
        high_conf_wrong = torch.tensor(0.0, device=probs.device)

    conf_wrong_penalty = (confidence * wrong * sample_weights).mean() + high_conf_wrong

    loss = bce + lambda_penalty * conf_wrong_penalty
    loss.backward()
    optimizer.step()

print("Weights:", model[0].weight.data)
print("Bias:", model[0].bias.data)

Weights: tensor([[ 0.0099, -0.0755, -0.0299, -0.0260,  0.1496, -0.3189, -0.4328,  0.4441,
          0.2995, -0.0502, -0.0104,  0.0623, -0.0100, -0.0048,  0.0471, -0.1188,
          0.1738,  0.0738, -0.0535, -0.0439, -0.0216]])
Bias: tensor([-0.0690])


In [13]:
# regression model to predict points directly
reg_model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1)
)
reg_criterion = nn.MSELoss(reduction='none')
reg_optimizer = torch.optim.Adam(reg_model.parameters(), lr=0.01, weight_decay=1e-4)

reg_epochs = 1000

for _ in range(reg_epochs):
    reg_optimizer.zero_grad()
    pts_pred = reg_model(X_train)
    mse_per_sample = reg_criterion(pts_pred, y_pts_train)
    # reuse same recency weights as classification
    reg_loss = (mse_per_sample * sample_weights).mean()
    reg_loss.backward()
    reg_optimizer.step()

print("Regression weights:", reg_model[0].weight.data)
print("Regression bias:", reg_model[0].bias.data)

Regression weights: tensor([[ 0.0984, -0.1195,  0.0830,  0.0467,  0.3072,  0.0984,  0.1540,  0.2231,
          0.3378, -0.6510, -0.2725,  0.6931, -0.0432,  0.0019,  0.1442,  0.5517,
          0.8516, -0.0021,  0.1245,  0.0629,  0.1424]])
Regression bias: tensor([0.2702])


In [14]:
import numpy as np

# y_test is 0/1, y_pred is probabilities, actual_above_below already built
with torch.no_grad():
    y_pred = model(X_test)

def interpret_prediction(prob):
    if prob >= 0.80:
        return "High confidence above average"
    elif prob >= 0.60:
        return "Moderate confidence above average"
    elif prob > 0.40:
        return "Uncertain, could be average"
    elif prob <= 0.4:
        return "Moderate confidence below average"
    elif prob <= 0.20:
        return "High confidence below average"
    else:
        return "High confidence below average"

labels = []
correct = []

for i, prob in enumerate(y_pred):
    p = prob.item()
    lbl = interpret_prediction(p)
    labels.append(lbl)

    # model's hard prediction: above if p >= 0.5
    pred_cls = 1.0 if p >= 0.5 else 0.0
    true_cls = y_test[i].item()
    correct.append(1 if pred_cls == true_cls else 0)

labels = np.array(labels)
correct = np.array(correct)

# counters for right and wrong decisions (overall)
total_predictions = len(correct)
total_correct = int(correct.sum())
total_wrong = int(total_predictions - total_correct)
print(f"\nModel decisions: {total_correct} correct, {total_wrong} wrong out of {total_predictions}")

# split counters by confidence level (high, moderate, low)
high_total = high_correct = 0
moderate_total = moderate_correct = 0
low_total = low_correct = 0

for lbl, is_corr in zip(labels, correct):
    if lbl.startswith("High confidence"):
        high_total += 1
        if is_corr:
            high_correct += 1
    elif lbl.startswith("Moderate confidence"):
        moderate_total += 1
        if is_corr:
            moderate_correct += 1
    else:  # treat everything else as low confidence / uncertain
        low_total += 1
        if is_corr:
            low_correct += 1

def _print_conf_summary(name, total, correct):
    if total == 0:
        print(f"{name}: no examples")
    else:
        wrong = total - correct
        acc = correct / total
        print(f"{name}: {correct} correct, {wrong} wrong out of {total} (acc={acc:5.2f})")

print("\nBy confidence level:")
_print_conf_summary("High confidence", high_total, high_correct)
_print_conf_summary("Moderate confidence", moderate_total, moderate_correct)
_print_conf_summary("Low confidence", low_total, low_correct)

print("\nCalibration per verbal label:")
for lbl in sorted(set(labels)):
    mask = labels == lbl
    n = mask.sum()
    acc = correct[mask].mean() if n > 0 else float("nan")
    print(f"{lbl:35s} n={n:2d}, accuracy={acc:5.2f}")


Model decisions: 17 correct, 13 wrong out of 30

By confidence level:
High confidence: no examples
Moderate confidence: 12 correct, 2 wrong out of 14 (acc= 0.86)
Low confidence: 5 correct, 11 wrong out of 16 (acc= 0.31)

Calibration per verbal label:
Moderate confidence above average   n= 1, accuracy= 1.00
Moderate confidence below average   n=13, accuracy= 0.85
Uncertain, could be average         n=16, accuracy= 0.31


In [15]:
# hardcoded to test today's game
X_test_np_today = [[36.6, 23, 7, 4, .522, .143, .688, .543, 1.000, 4, 5, 5, 16.5, 2, 29, 1, 0, 114.0, 119.7, -5.7, 96.9]]
X_test_today = torch.tensor(X_test_np_today, dtype=torch.float32)
y_today = model(X_test_today)

# above average vs. below average calculation
predicted_above_below = []
actual_above_below = []
for pred in y_today:
    if pred.item() > 0.5:
        predicted_above_below.append("Above Average")
    else:
        predicted_above_below.append("Below Average")

for actual in y_test:
    if actual.item() > 0.5:
        actual_above_below.append("Above Average")
    else:
        actual_above_below.append("Below Average")

print(interpret_prediction(y_today.item()))
y_today.item()

Uncertain, could be average


0.4526937007904053

In [16]:
interpreted_data = []
for pred in y_pred:
    interpreted_data.append(interpret_prediction(pred.item()))
    print(str(pred.item()) + " - "  + interpret_prediction(pred.item()) + " - " + str(actual_above_below[len(interpreted_data)-1]))

0.6068565845489502 - Moderate confidence above average - Above Average
0.5151800513267517 - Uncertain, could be average - Below Average
0.3709820508956909 - Moderate confidence below average - Below Average
0.44218218326568604 - Uncertain, could be average - Above Average
0.422963410615921 - Uncertain, could be average - Above Average
0.30150699615478516 - Moderate confidence below average - Below Average
0.49167194962501526 - Uncertain, could be average - Above Average
0.2954097092151642 - Moderate confidence below average - Below Average
0.3839735984802246 - Moderate confidence below average - Below Average
0.3854684829711914 - Moderate confidence below average - Above Average
0.4051227569580078 - Uncertain, could be average - Above Average
0.32889705896377563 - Moderate confidence below average - Below Average
0.5125371813774109 - Uncertain, could be average - Below Average
0.43428835272789 - Uncertain, could be average - Below Average
0.3070039451122284 - Moderate confidence below 